In [23]:
from sympy import symbols, Matrix
from sympy import expand, factor #expand and factor expressions
import random
import numpy as np
from sympy import Matrix,eye

[Sympy doc] (https://docs.sympy.org/latest/tutorials/intro-tutorial/index.html)

1. Symbolic computation: manipulating irrational numbers exactly using SymPy, computing symbolic expressions with variables.
2. most simplifications in Sympy are not performed automatically.
3. SymPy can simplify expressions, compute derivatives, integrals, and limits, solve equations, work with matrices, and much, much more, and do it all symbolically.



In [4]:
x,y=symbols('x y')
expr=x+2*y
expanded_expr=expand(x*expr)
factor(expanded_expr)

x*(x + 2*y)

some rules:
* To define variables, we must use symbols.
* if a variable is changed, expressions that were already created with that variable do not change automatically
* == represents exact structural equality testing. “Exact” here means that two expressions will compare equal with == only if they are exactly equal structurally.
* the best way to check whether a-b: subtract, simplify(b-a), and see whether you get 0 or a.equals(b)

In [33]:
x = symbols('x')
expr = x + 1
# x = 2
#print(expr)#changes made to x has no effects in expr, which is created before the chg
expr.subs(x, 2)

3

Converting Strings to SymPy Expressions
**sympify**, not to be confused with simplify

# Key Generation

In [34]:
def poly_n_variables(n):
    K=[]
    for i in range(n):
        x='x'+str(i)
        K.append(x)
    return K

def sparse_polynomial(n,t,b,q):#t monomials, bound the degree of monomials by an integer b
    K=poly_n_variables(n)
    sparse_poly=0
    
    for i in range(t):
        deg=random.randint(1,b)
        monomoial_d=1
        
        for i in range(deg):
            monomoial_d*=symbols(random.choice(K))
        sparse_poly+=random.randint(1,q-1)*monomoial_d
    return sparse_poly

sparse_polynomial(32,5,6,6)

x0*x13*x16*x23*x7 + 3*x10*x27 + 5*x12*x2*x8 + 2*x2 + 3*x25*x3*x6*x7

# Generate the matrix

In [94]:
def generate_U(k,n,t,B,q):
    I=eye(k)
    
    for i in range(int((k*k-k)/2)):
        b=random.randint(1,k-1)
        a=random.randint(0,b-1)
        I[a,b]=sparse_polynomial(n,t,B,q)
    return I
    
def generate_L(k,n,t,B,q):
    I=eye(k)
    
    for i in range(int((k*k-k)/2)):
        a=random.randint(1,k-1)
        b=random.randint(0,a-1)
        I[a,b]=sparse_polynomial(n,t,B,q)
    return I

generate_U(4,32,5,6,6)

Matrix([
[1, 2*x12*x14*x2*x26 + 4*x14*x15**2*x21*x3 + 5*x14*x25*x26 + 4*x20 + 2*x21*x22*x25*x29*x3,                  4*x10*x16*x20*x26**2 + 5*x10*x8 + 5*x11*x22*x6 + 5*x2 + 3*x21*x28, 0],
[0,                                                                                     1, 3*x0*x2*x23 + 3*x14*x15*x2**2 + 4*x14*x19**2*x20*x29*x5 + x16*x19*x24*x30*x9 + x26, 0],
[0,                                                                                     0,                                                                                  1, 0],
[0,                                                                                     0,                                                                                  0, 1]])

In [70]:

def col_permutation(matrix):
    col_permutation = random.sample(range(matrix.cols), matrix.cols)
    permuted_matrix = matrix.permute_cols(col_permutation)
    
    return permuted_matrix
col_permutation(Matrix([[1, 4, 7], [2, 5, 8], [3, 6, 9]]))

Matrix([
[4, 7, 1],
[5, 8, 2],
[6, 9, 3]])

In [86]:

def random_del_columns(k,l,matrix):
    col_indices = random.sample(range(matrix.cols), k-l)#ramdonly rm k-l cols
    for i in col_indices:
        matrix.col_del(i)

    return matrix

random_del_columns(3,2,Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]]))

Matrix([
[2, 3],
[5, 6],
[8, 9]])

In [103]:
def multiply_out(sample_M):
    for i in range(sample_M.rows):
        for j in range(sample_M.cols):
            sample_M[i,j]=expand(sample_M[i,j])
    return sample_M

multiply_out(Matrix([[1, 4, 7], [2, 5, 8], [3, 6, 9]]))

Matrix([
[1, 4, 7],
[2, 5, 8],
[3, 6, 9]])

In [111]:
def matrix_kl(k,l,s,n,t,B,q):#k,l are the dimensions of the matrix, s is the several, a small number
    Ls=[]
    Us=[]
    for i in range(s):
        U=generate_U(k,n,t,B,q)
        L=generate_L(k,n,t,B,q)
        Us.append(U)
        Ls.append(L)
        
    U_new=random.choice(Us)
    L_new=random.choice(Ls)
    Ms=Us+Ls
    
    T1=random.choice(Ms)
    P1=col_permutation(T1)
    T2=random.choice(Ms)
    P2=col_permutation(T2)
    T3=random.choice(Ms)
    P3=col_permutation(T3)
    kk_matrix=U_new*L_new*P1*T1*P2*T2*P3
    
    kl_matrix=random_del_columns(k,l,kk_matrix)
    return kl_matrix

print(matrix_kl(4,3,4,32,3,4,6)[1,1])



3*x10*x7 + 4*x18*x9 + x20*x21*x23*x25 + ((3*x10*x7 + 4*x18*x9 + x20*x21*x23*x25)*(5*x1*x12*x22*x5 + 3*x1 + 3*x10*x14*x29*x5) + 2*(3*x10*x7 + 4*x18*x9 + x20*x21*x23*x25)*(3*x15*x26*x30*x4 + 2*x24 + 4*x28*x5) + (5*x1*x12*x22*x5 + 3*x1 + 3*x10*x14*x29*x5)*(2*x11*x15*x29*x8 + 5*x17*x22*x3*x6 + 4*x30*x4*x6))*(4*x10*x2*x26*x3 + x10*x24 + 5*x27) + (3*x15*x26*x30*x4 + 2*x24 + 4*x28*x5)*(((3*x10*x7 + 4*x18*x9 + x20*x21*x23*x25)*(3*x15*x26*x30*x4 + 2*x24 + 4*x28*x5) + (5*x1*x12*x22*x5 + 3*x1 + 3*x10*x14*x29*x5)*(2*x11*x15*x29*x8 + 5*x17*x22*x3*x6 + 4*x30*x4*x6))*(5*x1*x12*x22*x5 + 3*x1 + 3*x10*x14*x29*x5) + (x1 + 5*x15*x27*x8 + x31)*(3*x10*x7 + 4*x18*x9 + x20*x21*x23*x25) + (5*x1*x12*x22*x5 + 3*x1 + 3*x10*x14*x29*x5)**2*(2*x11*x15*x29*x8 + 5*x17*x22*x3*x6 + 4*x30*x4*x6) + 1)
